# Configs

[![Github](https://img.shields.io/github/stars/lab-ml/labml?style=social)](https://github.com/lab-ml/labml)
[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/lab-ml/labml/blob/master/guides/configs.ipynb)
[![Docs](https://img.shields.io/badge/labml-docs-blue)](https://docs.labml.ai/api/configs.html)

The configurations provide an API to easily manage hyper-parameters
and other configurable parameters of the experiments.
The configuration of each experiment run are stored.
These can be viewed on [the web app](https://github.com/labmlai/labml/tree/master/app).

In [ ]:
!pip install labml --quiet

In [ ]:
import torch
from torch import nn

from labml import tracker, monit, experiment, logger
from labml.configs import BaseConfigs, option, calculate, hyperparams, aggregate

#### Define a configuration class

In [ ]:
class TransformerConfigs(BaseConfigs):
    d_model: int = 512
    d_ff: int = 2048
    attention: nn.Module = 'MultiHead'
    ffn: nn.Module = 'MLP'
    ffn_activation: nn.Module = 'ReLU'

Use of type hinting is optional.

#### Calculated configurations

You can specify multiple config calculator functions.
You pick which one to use by its name.

In [ ]:
@option(TransformerConfigs.ffn_activation)
def ReLU(c: TransformerConfigs):
    return nn.ReLU()

@option(TransformerConfigs.ffn_activation)
def GELU(c: TransformerConfigs):
    return nn.GELU()

#### Inheriting and re-using configuration classes

Configs classes can be inherited. This lets you separate configs into modules instead of passing [monolithic config object](https://www.reddit.com/r/MachineLearning/comments/g1vku4/d_antipatterns_in_open_sourced_ml_research_code/).

You can even inherit a entire experiment setups and make a few modifications.

In [ ]:
class MyTransformerConfigs(TransformerConfigs):
    positional_embeddings: nn.Module = 'Rotary'
    ffn_activation: nn.Module = 'GELU'

#### Submodules

Configurations can be nested.

In [ ]:
class Configs(BaseConfigs):
    transformer: TransformerConfigs = 'rotary_transformer'
    
    total_steps: int
    epochs: int
    steps_per_epoch: int
    
    tokenizer: any
    dataset: any
    
    task: any
    
@option(Configs.transformer, 'rotary_transformer')
def rotary_transformer_configs(c: Configs):
    conf = MyTransformerConfigs()
    
    conf.d_model = 256
    
    return conf

*It will initialize to default (based on type hint) if no options are provided.*

### Advanced Usage

#### Calculating with predefined functions or lambdas

You can also compute configs with `lambda` functions or predefined functions

In [ ]:
_ = calculate(Configs.total_steps, 
          [Configs.epochs, Configs.steps_per_epoch], # args
          lambda e, s: e * s)

#### Aggregates

You can use aggregates to setup configs that depend on each other.

For example, we change `dataset` and `epochs` based on the `task`.

In [ ]:
aggregate(Configs.task, 'wiki', (Configs.dataset, 'wikipedia'), (Configs.epochs, 10))
aggregate(Configs.task, 'arxiv', (Configs.dataset, 'arxiv'), (Configs.epochs, 100))

#### Hyper-parameters

labml will identify any parameter you modify outside the declaration
of the class as hyper-parameters.
You can also specify hyper-parameters manually.

The hyper-parameters will be highlighted among other configs in logs and in [the web app](https://github.com/labmlai/labml/tree/master/app).

These will also be logged in to Tensorboard.

In [ ]:
hyperparams(Configs.epochs)
hyperparams(Configs.total_steps, is_hyperparam=False)

#### Running the experiment

Here's how you run an experiment with the configurations.

In [ ]:
conf = Configs()
conf.task = 'arxiv'
experiment.create(name='test_configs')
experiment.configs(conf)
logger.inspect(model=conf.epochs)

In [ ]:
experiment.start()